# Part 1

Implement t-test for 2 iid distributions. Calculate mean, variance and p-value for different hypothesis (one-sided, two-sided), and result (if hypothesis rejected or not). Compare resuts with scipy.stats.ttest_ind

In [5]:
import numpy as np
from scipy.stats import t

def t_test_2samp(data1, data2, alpha, alternative='two-sided'):
    # Calculate mean and variance for each sample
    mean1 = np.mean(data1)
    mean2 = np.mean(data2)
    var1 = np.var(data1, ddof=1)
    var2 = np.var(data2, ddof=1)
    
    # Calculate the test statistic
    n1 = len(data1)
    n2 = len(data2)
    pooled_var = ((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2)
    t_stat = (mean1 - mean2) / np.sqrt(pooled_var * (1/n1 + 1/n2))
    
    # Calculate degrees of freedom
    df = n1 + n2 - 2
    
    # Calculate p-value
    if alternative == 'two-sided':
        # Calculate the cumulative distribution function (CDF) of the t-distribution
        cdf = 1 - (1 + np.abs(t_stat)) / 2
        
        # Calculate the p-value based on the CDF
        p_value = 2 * min(cdf, 1 - cdf)
    else:
        # Calculate the cumulative distribution function (CDF) of the t-distribution
        cdf = 1 - t.cdf(np.abs(t_stat), df)
        
        # Calculate the p-value based on the CDF
        if alternative == 'less':
            p_value = cdf
        elif alternative == 'greater':
            p_value = 1 - cdf
        else:
            raise ValueError("Invalid alternative hypothesis specified.")
    

    
    # Determine if the null hypothesis is rejected based on the p-value
    if p_value < alpha:
        hypothesis_result = 'Rejected'
    else:
        hypothesis_result = 'Not Rejected'
    
    return mean1, mean2, var1, var2, t_stat, p_value, hypothesis_result


In [6]:
from scipy.stats import ttest_ind

# Generate two samples from normal distributions
np.random.seed(42)
data1 = np.random.normal(loc=0, scale=1, size=100)
data2 = np.random.normal(loc=0.5, scale=1, size=100)

# Perform t-test and print the results
alpha = 0.05

# Two-sided hypothesis
mean1, mean2, var1, var2, t_stat, p_value, hypothesis_result = t_test_2samp(data1, data2, alpha)
t_stat_skipy, pvalue_skipy = ttest_ind(data1, data2, alternative='two-sided')

print("Two-Sided Hypothesis:")
print("Mean 1:", mean1)
print("Mean 2:", mean2)
print("Variance 1:", var1)
print("Variance 2:", var2)
print("T-Statistic implemented VS T-Statistic scipy.stats:", t_stat, t_stat_skipy)
print("P-Value implemented VS P-Value scipy.stats:", p_value, pvalue_skipy)
print("Hypothesis Result:", hypothesis_result)

# One-sided hypothesis (alternative='less')
mean1, mean2, var1, var2, t_stat, p_value, hypothesis_result = t_test_2samp(data1, data2, alpha, alternative='less')
t_stat_skipy, pvalue_skipy = ttest_ind(data1, data2, alternative='less')
print("\nOne-Sided Hypothesis (Alternative: Less):")
print("Mean 1:", mean1)
print("Mean 2:", mean2)
print("Variance 1:", var1)
print("Variance 2:", var2)
print("T-Statistic implemented VS T-Statistic scipy.stats:", t_stat, t_stat_skipy)
print("P-Value implemented VS P-Value scipy.stats:", p_value, pvalue_skipy)
print("Hypothesis Result:", hypothesis_result)

# One-sided hypothesis (alternative='greater')
mean1, mean2, var1, var2, t_stat, p_value, hypothesis_result = t_test_2samp(data1, data2, alpha, alternative='greater')
t_stat_skipy, pvalue_skipy = ttest_ind(data1, data2, alternative='greater')
print("\nOne-Sided Hypothesis (Alternative: Greater):")
print("Mean 1:", mean1)
print("Mean 2:", mean2)
print("Variance 1:", var1)
print("Variance 2:", var2)
print("T-Statistic implemented VS T-Statistic scipy.stats:", t_stat, t_stat_skipy)
print("P-Value implemented VS P-Value scipy.stats:", p_value, pvalue_skipy)
print("Hypothesis Result:", hypothesis_result)


Two-Sided Hypothesis:
Mean 1: -0.10384651739409384
Mean 2: 0.5223045870499239
Variance 1: 0.82476989363016
Variance 2: 0.9094844970607499
T-Statistic implemented VS T-Statistic scipy.stats: -4.754695943505288 -4.754695943505288
P-Value implemented VS P-Value scipy.stats: -3.754695943505288 3.819135262679343e-06
Hypothesis Result: Rejected

One-Sided Hypothesis (Alternative: Less):
Mean 1: -0.10384651739409384
Mean 2: 0.5223045870499239
Variance 1: 0.82476989363016
Variance 2: 0.9094844970607499
T-Statistic implemented VS T-Statistic scipy.stats: -4.754695943505288 -4.754695943505288
P-Value implemented VS P-Value scipy.stats: 1.909567631352971e-06 1.9095676313396715e-06
Hypothesis Result: Rejected

One-Sided Hypothesis (Alternative: Greater):
Mean 1: -0.10384651739409384
Mean 2: 0.5223045870499239
Variance 1: 0.82476989363016
Variance 2: 0.9094844970607499
T-Statistic implemented VS T-Statistic scipy.stats: -4.754695943505288 -4.754695943505288
P-Value implemented VS P-Value scipy.stat

In [6]:
ttest_ind(data1, data2)

Ttest_indResult(statistic=-4.754695943505288, pvalue=3.819135262679343e-06)

# Part 2

Implement bootstrap for estimating using confidence intervals median and mean values of distribution.  Calculate for distributions: normal, exponential, normal mixture

In [8]:
import numpy as np

def bootstrap(data, statistic, n_bootstrap, alpha):
    n = len(data)
    boot_samples = np.random.choice(data, size=(n_bootstrap, n), replace=True)
    boot_statistics = np.zeros(n_bootstrap)
    
    for i in range(n_bootstrap):
        boot_statistics[i] = statistic(boot_samples[i])
    
    boot_statistics.sort()
    lower_idx = int(n_bootstrap * alpha / 2)
    upper_idx = int(n_bootstrap * (1 - alpha / 2))
    lower_bound = boot_statistics[lower_idx]
    upper_bound = boot_statistics[upper_idx]
    
    return lower_bound, upper_bound


In [9]:
# Normal Distribution
np.random.seed(42)
normal_data = np.random.normal(loc=0, scale=1, size=1000)

mean_lower, mean_upper = bootstrap(normal_data, np.mean, n_bootstrap=1000, alpha=0.05)
print("Bootstrap confidence interval for mean (Normal distribution):")
print("Lower bound:", mean_lower)
print("Upper bound:", mean_upper)

median_lower, median_upper = bootstrap(normal_data, np.median, n_bootstrap=1000, alpha=0.05)
print("\nBootstrap confidence interval for median (Normal distribution):")
print("Lower bound:", median_lower)
print("Upper bound:", median_upper)

# Exponential Distribution
exponential_data = np.random.exponential(scale=1, size=1000)

mean_lower, mean_upper = bootstrap(exponential_data, np.mean, n_bootstrap=1000, alpha=0.05)
print("\nBootstrap confidence interval for mean (Exponential distribution):")
print("Lower bound:", mean_lower)
print("Upper bound:", mean_upper)

median_lower, median_upper = bootstrap(exponential_data, np.median, n_bootstrap=1000, alpha=0.05)
print("\nBootstrap confidence interval for median (Exponential distribution):")
print("Lower bound:", median_lower)
print("Upper bound:", median_upper)

# Normal Mixture Distribution
np.random.seed(42)
normal_mixture_data = np.concatenate([np.random.normal(loc=-2, scale=1, size=500),
                                      np.random.normal(loc=2, scale=1, size=500)])

mean_lower, mean_upper = bootstrap(normal_mixture_data, np.mean, n_bootstrap=1000, alpha=0.05)
print("\nBootstrap confidence interval for mean (Normal Mixture distribution):")
print("Lower bound:", mean_lower)
print("Upper bound:", mean_upper)

median_lower, median_upper = bootstrap(normal_mixture_data, np.median, n_bootstrap=1000, alpha=0.05)
print("\nBootstrap confidence interval for median (Normal Mixture distribution):")
print("Lower bound:", median_lower)
print("Upper bound:", median_upper)


Bootstrap confidence interval for mean (Normal distribution):
Lower bound: -0.03927738880320801
Upper bound: 0.08548538950280042

Bootstrap confidence interval for median (Normal distribution):
Lower bound: -0.0485875328972064
Upper bound: 0.09179287623280905

Bootstrap confidence interval for mean (Exponential distribution):
Lower bound: 0.9267077491235503
Upper bound: 1.0473770354688248

Bootstrap confidence interval for median (Exponential distribution):
Lower bound: 0.6264639404520238
Upper bound: 0.7242315740293506

Bootstrap confidence interval for mean (Normal Mixture distribution):
Lower bound: -0.11866773660851121
Upper bound: 0.15436030754632063

Bootstrap confidence interval for median (Normal Mixture distribution):
Lower bound: -0.4716445001272893
Upper bound: 0.5808566015820653


# Part 3

Calculate power of t-test and Mann-Whitney test for different distributions and effect.  Separately cover case when means are equals and variances are very different

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu

def calculate_power(effect_sizes, sample_size, n_simulations, alpha):
    power_ttest = []
    power_mannwhitney = []

    for effect_size in effect_sizes:
        n_rejections_ttest = 0
        n_rejections_mannwhitney = 0

        for _ in range(n_simulations):
            # Generate two samples with specified effect size
            sample1 = np.random.normal(loc=0, scale=1, size=sample_size)
            sample2 = np.random.normal(loc=effect_size, scale=1, size=sample_size)

            # Perform t-test
            _, p_value_ttest = ttest_ind(sample1, sample2)
            if p_value_ttest < alpha:
                n_rejections_ttest += 1

            # Perform Mann-Whitney test
            _, p_value_mannwhitney = mannwhitneyu(sample1, sample2, alternative='two-sided')
            if p_value_mannwhitney < alpha:
                n_rejections_mannwhitney += 1

        # Calculate power as the proportion of rejections
        power_ttest.append(n_rejections_ttest / n_simulations)
        power_mannwhitney.append(n_rejections_mannwhitney / n_simulations)

    return power_ttest, power_mannwhitney

In [10]:
import numpy as np

# Scenario 1: Different distributions with different means and equal variances
effect_sizes = np.linspace(0, 1, 11)
sample_size = 100
n_simulations = 1000
alpha = 0.05

power_ttest, power_mannwhitney = calculate_power(effect_sizes, sample_size, n_simulations, alpha)

print("Scenario 1: Different distributions with different means and equal variances")
print("Effect Sizes:", effect_sizes)
print("Power (t-test):", power_ttest)
print("Power (Mann-Whitney):", power_mannwhitney)

# Scenario 2: Different distributions with equal means and different variances
effect_sizes = np.linspace(0, 1, 11)
sample_size = 100
n_simulations = 1000
alpha = 0.05

power_ttest, power_mannwhitney = calculate_power(effect_sizes, sample_size, n_simulations, alpha)

print("\nScenario 2: Different distributions with equal means and different variances")
print("Effect Sizes:", effect_sizes)
print("Power (t-test):", power_ttest)
print("Power (Mann-Whitney):", power_mannwhitney)

# Scenario 3: Same distribution with equal means and variances
effect_sizes = np.linspace(0, 1, 11)
sample_size = 100
n_simulations = 1000
alpha = 0.05

power_ttest, power_mannwhitney = calculate_power(effect_sizes, sample_size, n_simulations, alpha)

print("\nScenario 3: Same distribution with equal means and variances")
print("Effect Sizes:", effect_sizes)
print("Power (t-test):", power_ttest)
print("Power (Mann-Whitney):", power_mannwhitney)


Scenario 1: Different distributions with different means and equal variances
Effect Sizes: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Power (t-test): [0.057, 0.115, 0.288, 0.551, 0.834, 0.938, 0.987, 0.998, 0.999, 1.0, 1.0]
Power (Mann-Whitney): [0.058, 0.114, 0.276, 0.538, 0.81, 0.932, 0.976, 0.998, 0.999, 1.0, 1.0]

Scenario 2: Different distributions with equal means and different variances
Effect Sizes: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Power (t-test): [0.032, 0.099, 0.325, 0.561, 0.793, 0.944, 0.988, 0.999, 1.0, 1.0, 1.0]
Power (Mann-Whitney): [0.036, 0.096, 0.318, 0.544, 0.771, 0.93, 0.982, 0.999, 1.0, 1.0, 1.0]

Scenario 3: Same distribution with equal means and variances
Effect Sizes: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Power (t-test): [0.052, 0.098, 0.303, 0.535, 0.791, 0.935, 0.992, 1.0, 1.0, 1.0, 1.0]
Power (Mann-Whitney): [0.052, 0.099, 0.28, 0.521, 0.771, 0.923, 0.987, 1.0, 1.0, 1.0, 1.0]


# Part 4

In [8]:
from scipy.stats import ttest_ind, mannwhitneyu

def test_correctness(distribution, sample_size, alpha):
    # Generate samples from the specified distribution
    if distribution == 'normal':
        sample1 = np.random.normal(loc=0, scale=1, size=sample_size)
        sample2 = np.random.normal(loc=0, scale=1, size=sample_size)
    elif distribution == 'exponential':
        sample1 = np.random.exponential(scale=1, size=sample_size)
        sample2 = np.random.exponential(scale=1, size=sample_size)
    elif distribution == 'mixture':
        sample1 = np.concatenate([
            np.random.normal(loc=0, scale=1, size=int(sample_size/2)),
            np.random.normal(loc=5, scale=1, size=int(sample_size/2))
        ])
        sample2 = np.concatenate([
            np.random.normal(loc=0, scale=1, size=int(sample_size/2)),
            np.random.normal(loc=5, scale=1, size=int(sample_size/2))
        ])

    # Perform t-test
    t_stat, p_value_ttest = ttest_ind(sample1, sample2)

    # Perform Mann-Whitney test
    _, p_value_mannwhitney = mannwhitneyu(sample1, sample2, alternative='two-sided')

    # Determine correctness of the tests
    ttest_correct = (p_value_ttest >= alpha)
    mannwhitney_correct = (p_value_mannwhitney >= alpha)

    return ttest_correct, mannwhitney_correct

In [9]:
# Parameters
sample_size = 100
alpha = 0.05

# Test correctness on different distributions
distributions = ['normal', 'exponential', 'mixture']
n_tests = 1000

for distribution in distributions:
    ttest_correct_count = 0
    mannwhitney_correct_count = 0

    for _ in range(n_tests):
        ttest_correct, mannwhitney_correct = test_correctness(distribution, sample_size, alpha)
        ttest_correct_count += ttest_correct
        mannwhitney_correct_count += mannwhitney_correct

    ttest_correct_percentage = (ttest_correct_count / n_tests) * 100
    mannwhitney_correct_percentage = (mannwhitney_correct_count / n_tests) * 100

    print(f"\nDistribution: {distribution.capitalize()}")
    print("T-Test Correctness: {:.2f}%".format(ttest_correct_percentage))
    print("Mann-Whitney Correctness: {:.2f}%".format(mannwhitney_correct_percentage))



Distribution: Normal
T-Test Correctness: 94.00%
Mann-Whitney Correctness: 94.00%

Distribution: Exponential
T-Test Correctness: 95.10%
Mann-Whitney Correctness: 94.70%

Distribution: Mixture
T-Test Correctness: 100.00%
Mann-Whitney Correctness: 100.00%
